# Set Up

In [1]:
# Imports and Helper Functions
import pandas as pd
import RNA
import os
import numpy as np

In [2]:
def dotbracket_to_pairset(db_string: str) -> set:
    """Converts a dot-bracket string to a set of 1-indexed base pairs."""
    pairs = set()
    stack = []
    for i, char in enumerate(db_string):
        # Use 1-based indexing for biological convention
        pos = i + 1
        if char == '(':
            stack.append(pos)
        elif char == ')':
            if stack:
                j = stack.pop()
                # Store pair as (smaller_index, larger_index)
                pairs.add((j, pos))
    return pairs

# def parse_target_region(region_string: str) -> list:
#     """Converts a region string like '50-52' or '50' to a list of integers."""
#     if not region_string or pd.isna(region_string):
#         return []
#     region_string = str(region_string)
#     if '-' in region_string:
#         start, end = map(int, region_string.split('-'))
#         return list(range(start, end + 1))
#     else:
#         return [int(region_string)]

def parse_target_region(region_string) -> list:
    """
    Parses a complex region string like '50', '50-52', or '56-63, 69-81'
    into a list of all integer positions.
    """
    if pd.isna(region_string) or region_string == '':
        return []
    region_string = str(region_string).replace(' ', '')
    all_positions = []
    parts = region_string.split(',')
    for part in parts:
        try:
            if '-' in part:
                start, end = map(int, part.split('-'))
                all_positions.extend(range(start, end + 1))
            else:
                all_positions.append(int(part))
        except ValueError:
            print(f"Warning: Could not parse part of a region string: '{part}'")
            continue
    return all_positions

def set_to_str(s: set) -> str:
    """Converts a set of tuples to a sorted, human-readable string for CSV output."""
    if not s:
        return ""
    return "; ".join(map(str, sorted(list(s))))


In [3]:
# The Core Analysis Engine (Reworked with Correct Order of Operations)

def run_analysis_engine(
    mutant_ids: pd.Series,
    parent_seqs: pd.Series,
    mutant_seqs: pd.Series,
    parent_dbs: pd.Series,
    mutant_dbs: pd.Series,
    goal_types: pd.Series,
    target_regions_1: pd.Series
) -> pd.DataFrame:
    """Analyzes structural differences between parent and mutant RNAs.

    This robust function takes pandas Series of sequence/structure data, calculates
    distance metrics, and performs goal-specific analysis via substring matching.

    Args:
        mutant_ids (pd.Series): Unique string identifiers for each mutant.
        parent_seqs (pd.Series): Parent RNA sequences.
        mutant_seqs (pd.Series): Mutant RNA sequences.
        parent_dbs (pd.Series): Parent dot-bracket structure strings.
        mutant_dbs (pd.Series): Mutant dot-bracket structure strings.
        goal_types (pd.Series): Goal descriptions for substring matching. Keywords:
            "break_pair", "disruption", "extend_stem", "seed_hairpin",
            "compensatory", "stabilizing", "destabilizing".
        target_regions_1 (pd.Series): Primary target region, e.g., '50' or '50-52'.

    Returns:
        pd.DataFrame: A new DataFrame with analysis results, one row per mutant.
    """
    results = []
    
    num_entries = len(mutant_ids)
    print(f"Starting analysis of {num_entries} entries...")

    iterator = zip(
        mutant_ids, parent_seqs, mutant_seqs, parent_dbs, mutant_dbs, 
        goal_types, target_regions_1
    )

    for i, (mutant_id, p_seq, m_seq, p_db, m_db, goal, tr1_str) in enumerate(iterator):
        
        result_row = {'mutant_id': mutant_id, 'goal_type': goal, 'parent_seq': p_seq, 'mutant_seq': m_seq}

        target_region_1 = parse_target_region(tr1_str)
        # --- Core Metric Calculations ---
        
        parent_tree_repr = RNA.db_to_tree_string(p_db, RNA.STRUCTURE_TREE_SHAPIRO_WEIGHT)
        mutant_tree_repr = RNA.db_to_tree_string(m_db, RNA.STRUCTURE_TREE_SHAPIRO_WEIGHT)
        result_row['parent_tree_repr'] = parent_tree_repr
        result_row['mutant_tree_repr'] = mutant_tree_repr
        
        # Calculate Tree Edit Distance 
        tree_parent, tree_mutant = None, None
        try:
            # Create computational Tree objects 
            tree_parent = RNA.make_tree(parent_tree_repr)
            tree_mutant = RNA.make_tree(mutant_tree_repr)
            
            # Calculate distance using the valid tree objects.
            result_row['TreeEditDistance'] = RNA.tree_edit_distance(tree_parent, tree_mutant)
        finally:
            # Free the allocated memory.
            if tree_parent: RNA.free_tree(tree_parent)
            if tree_mutant: RNA.free_tree(tree_mutant)

        # Calculate Base Pair distance
        result_row['Full_BP_Distance'] = RNA.bp_distance(p_db, m_db)
        
        # --- Sequence & Pair Analysis ---
        mutations = [f"{p_seq[i]}{i+1}{m_seq[i]}" for i in range(len(p_seq)) if p_seq[i] != m_seq[i]]
        result_row['Sequence_Changes'] = "; ".join(mutations)
        
        p_parent_set = dotbracket_to_pairset(p_db)
        p_mutant_set = dotbracket_to_pairset(m_db)
        sym_diff = p_parent_set.symmetric_difference(p_mutant_set)
        
        changed_pairs_annotated = []
        for pair_i, pair_j in sorted(list(sym_diff)):
            if (pair_i, pair_j) in p_parent_set:
                pair_str = f"{p_seq[pair_i-1]}{pair_i}-{p_seq[pair_j-1]}{pair_j}"
                changed_pairs_annotated.append(f"Broke {pair_str}")
            elif (pair_i, pair_j) in p_mutant_set:
                pair_str = f"{m_seq[pair_i-1]}{pair_i}-{m_seq[pair_j-1]}{pair_j}"
                changed_pairs_annotated.append(f"Formed {pair_str}")
        result_row['Annotated_Pair_Changes'] = "; ".join(changed_pairs_annotated)
        
        # --- Goal-Specific Logic ---
        on_target_score, off_target_score, intended_changes, logic_type = "N/A", "N/A", set(), "unknown"
        
        if "break_pair" in goal:
            logic_type = "break_pair"
            on_target_score = sum(1 for pos in target_region_1 if m_db[pos - 1] == '.')
            intended_changes = {p for p in p_parent_set if p[0] in target_region_1 or p[1] in target_region_1}
            off_target_pairs = sym_diff - intended_changes
            off_target_score = len(off_target_pairs)

        elif "extend_stem" in goal or "seed_hairpin" in goal:
            logic_type = "make_pair"
            on_target_score = sum(1 for pos in target_region_1 if m_db[pos - 1] in '()')
            intended_changes = {p for p in p_mutant_set if p[0] in target_region_1 or p[1] in target_region_1}
            off_target_pairs = sym_diff - intended_changes
            off_target_score = len(off_target_pairs)

        elif "downstream_disruption" in goal or "upstream_disruption" in goal or "total_disruption" in goal:
            logic_type = "disruption"
            paired_in_target = sum(1 for pos in target_region_1 if m_db[pos - 1] != '.')
            on_target_score = paired_in_target
            intended_changes = {p for p in p_parent_set if p[0] in target_region_1 or p[1] in target_region_1}
            off_target_pairs = sym_diff - intended_changes
            off_target_score = len(off_target_pairs)

        elif "downstream_compensation" in goal or "upstream_compensation" in goal or "total_compensation" in goal:
            logic_type = "compensatory"
            # on_target_score = result_row['Full_BP_Distance']
            # off_target_score = len(sym_diff)

            parent_partner_map = {i: j for i, j in p_parent_set}
            parent_partner_map.update({j: i for i, j in p_parent_set})
            mutant_partner_map = {i: j for i, j in p_mutant_set}
            mutant_partner_map.update({j: i for i, j in p_mutant_set})

            restored_pair_count = 0
            for pos in target_region_1:
                original_partner = parent_partner_map.get(pos)
                mutant_partner = mutant_partner_map.get(pos)
                if original_partner is not None:
                    if mutant_partner == original_partner:
                        restored_pair_count += 1
                else:
                    if mutant_partner is None:
                        restored_pair_count += 1

            on_target_score = len(target_region_1) - restored_pair_count
            off_target_score = result_row['Full_BP_Distance']

        elif "stabilizing" in goal or "destabilizing" in goal:
            logic_type = "stability"
            on_target_score = "N/A"
            off_target_score = result_row['Full_BP_Distance']

        else:
            print(f"Warning: Goal '{goal}' for mutant '{mutant_id}' did not match any known keywords.")

        result_row['Logic_Type_Applied'] = logic_type
        result_row['OnTargetScore'] = on_target_score
        result_row['OffTargetScore'] = off_target_score
        result_row['IntendedChange_Pairs(positions)'] = set_to_str(intended_changes)
        
        results.append(result_row)
        
    # --- Finalize DataFrame ---
    output_df = pd.DataFrame(results)
    column_order = [
        'mutant_id', 'goal_type', 'Logic_Type_Applied', 'Sequence_Changes',
        'TreeEditDistance', 'OnTargetScore', 'OffTargetScore', 'Full_BP_Distance', 
        'parent_tree_repr', 'mutant_tree_repr', 'Annotated_Pair_Changes', 
        'IntendedChange_Pairs(positions)', "parent_seq", "mutant_seq"
    ]
    # Use reindex to safely order columns, filling missing ones with NaN
    output_df = output_df.reindex(columns=column_order)
    
    print("Analysis function finished.")
    return output_df

print("'run_analysis_engine' function is defined.")

'run_analysis_engine' function is defined.


In [4]:
# Define PUS7 Union

# Load the two CSV files into pandas DataFrames
incell_pus7_path = "C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool1_FINALANALYSIS/delpos/site_sum/PUS7dep_sites_fulllist.csv"
invitro_pus7_path = "C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/ivBID/202506/ivPUS7_sig_sites.csv"

incell_PUS7_df = pd.read_csv(incell_pus7_path)
invitro_PUS7_df = pd.read_csv(invitro_pus7_path)

PUS7_union_df = pd.merge(incell_PUS7_df, invitro_PUS7_df, on="chr", how="outer")

print(f"PUS7 data loaded and merged successfully. Number of sequences: {len(PUS7_union_df)}")


pool_path = "C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool1_Sequences/pool1_pos.xlsx"
pool_df = pd.read_excel(pool_path)

regex_pattern = r"^T[ACGT]TA[AG]$"
unuar_mask = pool_df['psipos_5mer'].str.contains(regex_pattern, regex=True, na=False)
unuar_df = pool_df.loc[unuar_mask]

print(f"UNUAR data loaded and merged successfully. Number of sequences: {len(unuar_df)}")

PUS7 data loaded and merged successfully. Number of sequences: 296
UNUAR data loaded and merged successfully. Number of sequences: 337


# Execution  

## Psi Pairing

In [5]:
unmod_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis.csv"

unmod_access_psi_df = pd.read_csv(unmod_access_psi)

column_mapping = {
    'mutant_ids': unmod_access_psi_df['name'],
    'parent_seqs': unmod_access_psi_df['original_seq'],
    'mutant_seqs': unmod_access_psi_df['mutated_seq'],
    'parent_dbs': unmod_access_psi_df['original_dot_bracket'],
    'mutant_dbs': unmod_access_psi_df['dot_bracket_mut'],
    'goal_types': unmod_access_psi_df['strategy'],
    'target_regions_1': unmod_access_psi_df['target_pos']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

break_pair_df = dfs_by_logic.get('break_pair')
make_pair_df = dfs_by_logic.get('make_pair')

print(f"Number of break pair: {len(break_pair_df)}")
print(f"Number of make pair: {len(make_pair_df)}")

break_pair_ted_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_ted_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

break_pair_off_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_off_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

good_break_pair_df = break_pair_df.loc[(break_pair_df['TreeEditDistance'] <= break_pair_ted_threshold) 
                                       & (break_pair_df['OnTargetScore'] == 1) 
                                       & (break_pair_df['OffTargetScore'] <= break_pair_off_threshold)]
good_make_pair_df = make_pair_df.loc[(make_pair_df['TreeEditDistance'] <= make_pair_ted_threshold) 
                                     & (make_pair_df['OnTargetScore'] == 1)
                                     & (make_pair_df['OffTargetScore'] <= make_pair_off_threshold)]

print(f"Number of good break pair: {len(good_break_pair_df)}")

good_break_pair_df.to_csv("unmod_good_breakpair_psi.csv", index=False)

print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 565 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 289 rows.
Creating DataFrame for logic type: 'make_pair' with 276 rows.
Number of break pair: 289
Number of make pair: 276
Number of good break pair: 7

Analysis complete. Results saved.


In [6]:
pool_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis_pool1.csv"

pool_access_psi_df = pd.read_csv(pool_access_psi)

column_mapping = {
    'mutant_ids': pool_access_psi_df['name'],
    'parent_seqs': pool_access_psi_df['original_seq'],
    'mutant_seqs': pool_access_psi_df['mutated_seq'],
    'parent_dbs': pool_access_psi_df['original_dot_bracket'],
    'mutant_dbs': pool_access_psi_df['dot_bracket_mut'],
    'goal_types': pool_access_psi_df['strategy'],
    'target_regions_1': pool_access_psi_df['target_pos']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

pool_break_pair_df = dfs_by_logic.get('break_pair')
pool_make_pair_df = dfs_by_logic.get('make_pair')

# The concise, single-line solution
poolPUS7_make_pair_df = pool_make_pair_df[pool_make_pair_df['mutant_id'].isin(PUS7_union_df['chr'])]
poolPUS7_break_pair_df = pool_break_pair_df[pool_break_pair_df['mutant_id'].isin(unuar_df['chr'])]

print(f"Number of break pair: {len(poolPUS7_break_pair_df)}")
print(f"Number of make pair: {len(poolPUS7_make_pair_df)}")

pool_break_pair_ted_threshold = poolPUS7_break_pair_df['TreeEditDistance'].quantile(0.5)
pool_make_pair_ted_threshold = poolPUS7_make_pair_df['TreeEditDistance'].quantile(0.5)

pool_break_pair_off_threshold = poolPUS7_break_pair_df['OffTargetScore'].quantile(0.5)
pool_make_pair_off_threshold = poolPUS7_make_pair_df['OffTargetScore'].quantile(0.5)

pool_good_break_pair_df = poolPUS7_break_pair_df.loc[(poolPUS7_break_pair_df['OnTargetScore'] == 1)
                                        & (poolPUS7_break_pair_df['TreeEditDistance'] <= pool_break_pair_ted_threshold) 
                                        & (poolPUS7_break_pair_df['OffTargetScore'] <= pool_break_pair_off_threshold)
                                        ]
pool_good_make_pair_df = poolPUS7_make_pair_df.loc[(poolPUS7_make_pair_df['OnTargetScore'] == 1)
                                        &  (poolPUS7_make_pair_df['TreeEditDistance'] <= pool_make_pair_ted_threshold) 
                                        & (poolPUS7_make_pair_df['OffTargetScore'] <= pool_make_pair_off_threshold)
                                        ]

print(f"Number of good break pair: {len(pool_good_break_pair_df)}")
print(f"Number of good make pair: {len(pool_good_make_pair_df)}")

pool_good_break_pair_df.to_csv("pool_good_breakpair_psi.csv", index=False)
pool_good_make_pair_df.to_csv("pool_good_makepair_psi.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 634 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 234 rows.
Creating DataFrame for logic type: 'make_pair' with 400 rows.
Number of break pair: 112
Number of make pair: 150
Number of good break pair: 31
Number of good make pair: 39

Analysis complete. Results saved.


## Psi +1 Pairing Accessibility

In [7]:
unmod_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis01.csv"

unmod_access_psi_df = pd.read_csv(unmod_access_psi)

column_mapping = {
    'mutant_ids': unmod_access_psi_df['name'],
    'parent_seqs': unmod_access_psi_df['original_seq'],
    'mutant_seqs': unmod_access_psi_df['mutated_seq'],
    'parent_dbs': unmod_access_psi_df['original_dot_bracket'],
    'mutant_dbs': unmod_access_psi_df['dot_bracket_mut'],
    'goal_types': unmod_access_psi_df['strategy'],
    'target_regions_1': unmod_access_psi_df['target_pos']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

break_pair_df = dfs_by_logic.get('break_pair')
make_pair_df = dfs_by_logic.get('make_pair')

print(f"Number of break pair: {len(break_pair_df)}")
print(f"Number of make pair: {len(make_pair_df)}")

break_pair_ted_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_ted_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

break_pair_off_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_off_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

good_break_pair_df = break_pair_df.loc[(break_pair_df['TreeEditDistance'] <= break_pair_ted_threshold) 
                                       & (break_pair_df['OnTargetScore'] == 2) 
                                       & (break_pair_df['OffTargetScore'] <= break_pair_off_threshold)]
good_make_pair_df = make_pair_df.loc[(make_pair_df['TreeEditDistance'] <= make_pair_ted_threshold) 
                                     & (make_pair_df['OnTargetScore'] == 2)
                                     & (make_pair_df['OffTargetScore'] <= make_pair_off_threshold)]

print(f"Number of good break pair: {len(good_break_pair_df)}")

good_break_pair_df.to_csv("unmod_good_breakpair_psi01.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 575 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 343 rows.
Creating DataFrame for logic type: 'make_pair' with 232 rows.
Number of break pair: 343
Number of make pair: 232
Number of good break pair: 4

Analysis complete. Results saved.


In [8]:
pool_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis01_pool1.csv"

pool_access_psi_df = pd.read_csv(pool_access_psi)

column_mapping = {
    'mutant_ids': pool_access_psi_df['name'],
    'parent_seqs': pool_access_psi_df['original_seq'],
    'mutant_seqs': pool_access_psi_df['mutated_seq'],
    'parent_dbs': pool_access_psi_df['original_dot_bracket'],
    'mutant_dbs': pool_access_psi_df['dot_bracket_mut'],
    'goal_types': pool_access_psi_df['strategy'],
    'target_regions_1': pool_access_psi_df['target_pos']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

pool_break_pair_df = dfs_by_logic.get('break_pair')
pool_make_pair_df = dfs_by_logic.get('make_pair')

# The concise, single-line solution
poolPUS7_make_pair_df = pool_make_pair_df[pool_make_pair_df['mutant_id'].isin(PUS7_union_df['chr'])]
poolPUS7_break_pair_df = pool_break_pair_df[pool_break_pair_df['mutant_id'].isin(unuar_df['chr'])]

print(f"Number of break pair: {len(poolPUS7_break_pair_df)}")
print(f"Number of make pair: {len(poolPUS7_make_pair_df)}")

pool_break_pair_ted_threshold = poolPUS7_break_pair_df['TreeEditDistance'].quantile(0.5)
pool_make_pair_ted_threshold = poolPUS7_make_pair_df['TreeEditDistance'].quantile(0.5)

pool_break_pair_off_threshold = poolPUS7_break_pair_df['OffTargetScore'].quantile(0.5)
pool_make_pair_off_threshold = poolPUS7_make_pair_df['OffTargetScore'].quantile(0.5)

pool_good_break_pair_df = poolPUS7_break_pair_df.loc[(poolPUS7_break_pair_df['OnTargetScore'] == 2)
                                        & (poolPUS7_break_pair_df['TreeEditDistance'] <= pool_break_pair_ted_threshold) 
                                        & (poolPUS7_break_pair_df['OffTargetScore'] <= pool_break_pair_off_threshold)
                                        ]
pool_good_make_pair_df = poolPUS7_make_pair_df.loc[(poolPUS7_make_pair_df['OnTargetScore'] == 2)
                                        &  (poolPUS7_make_pair_df['TreeEditDistance'] <= pool_make_pair_ted_threshold) 
                                        & (poolPUS7_make_pair_df['OffTargetScore'] <= pool_make_pair_off_threshold)
                                        ]

print(f"Number of good break pair: {len(pool_good_break_pair_df)}")
print(f"Number of good make pair: {len(pool_good_make_pair_df)}")

pool_good_break_pair_df.to_csv("pool_good_breakpair_psi01.csv", index=False)
pool_good_make_pair_df.to_csv("pool_good_makepair_psi01.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 644 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 306 rows.
Creating DataFrame for logic type: 'make_pair' with 338 rows.
Number of break pair: 148
Number of make pair: 126
Number of good break pair: 25
Number of good make pair: 35

Analysis complete. Results saved.


## Psipos 5mer Pairing Accessibility

In [9]:
unmod_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis5mer.csv"

unmod_access_psi_df = pd.read_csv(unmod_access_psi)

column_mapping = {
    'mutant_ids': unmod_access_psi_df['name'],
    'parent_seqs': unmod_access_psi_df['original_seq'],
    'mutant_seqs': unmod_access_psi_df['mutated_seq'],
    'parent_dbs': unmod_access_psi_df['original_dot_bracket'],
    'mutant_dbs': unmod_access_psi_df['dot_bracket_mut'],
    'goal_types': unmod_access_psi_df['strategy'],
    'target_regions_1': unmod_access_psi_df['target_pos_range']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

break_pair_df = dfs_by_logic.get('break_pair')
make_pair_df = dfs_by_logic.get('make_pair')

print(f"Number of break pair: {len(break_pair_df)}")
print(f"Number of make pair: {len(make_pair_df)}")

break_pair_ted_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_ted_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

break_pair_off_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_off_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

good_break_pair_df = break_pair_df.loc[(break_pair_df['OnTargetScore'] == 5)
                                       & (break_pair_df['TreeEditDistance'] <= break_pair_ted_threshold) 
                                       & (break_pair_df['OffTargetScore'] <= break_pair_off_threshold)
                                       ]
good_make_pair_df = make_pair_df.loc[(make_pair_df['OnTargetScore'] == 5)
                                     & (make_pair_df['TreeEditDistance'] <= make_pair_ted_threshold) 
                                     & (make_pair_df['OffTargetScore'] <= make_pair_off_threshold)
                                     ]

print(f"Number of good break pair: {len(good_break_pair_df)}")

good_break_pair_df.to_csv("unmod_good_breakpair_5mer.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 574 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 487 rows.
Creating DataFrame for logic type: 'make_pair' with 87 rows.
Number of break pair: 487
Number of make pair: 87
Number of good break pair: 12

Analysis complete. Results saved.


In [10]:
pool_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/psi_pairing/pairing_status_flip_analysis5mer_pool1.csv"

pool_access_psi_df = pd.read_csv(pool_access_psi)

column_mapping = {
    'mutant_ids': pool_access_psi_df['name'],
    'parent_seqs': pool_access_psi_df['original_seq'],
    'mutant_seqs': pool_access_psi_df['mutated_seq'],
    'parent_dbs': pool_access_psi_df['original_dot_bracket'],
    'mutant_dbs': pool_access_psi_df['dot_bracket_mut'],
    'goal_types': pool_access_psi_df['strategy'],
    'target_regions_1': pool_access_psi_df['target_pos_range']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

pool_break_pair_df = dfs_by_logic.get('break_pair')
pool_make_pair_df = dfs_by_logic.get('make_pair')

# The concise, single-line solution
poolPUS7_make_pair_df = pool_make_pair_df[pool_make_pair_df['mutant_id'].isin(PUS7_union_df['chr'])]
poolPUS7_break_pair_df = pool_break_pair_df[pool_break_pair_df['mutant_id'].isin(unuar_df['chr'])]

print(f"Number of break pair: {len(poolPUS7_break_pair_df)}")
print(f"Number of make pair: {len(poolPUS7_make_pair_df)}")

pool_break_pair_ted_threshold = poolPUS7_break_pair_df['TreeEditDistance'].quantile(0.5)
pool_make_pair_ted_threshold = poolPUS7_make_pair_df['TreeEditDistance'].quantile(0.5)

pool_break_pair_off_threshold = poolPUS7_break_pair_df['OffTargetScore'].quantile(0.5)
pool_make_pair_off_threshold = poolPUS7_make_pair_df['OffTargetScore'].quantile(0.5)

pool_good_break_pair_df = poolPUS7_break_pair_df.loc[(poolPUS7_break_pair_df['OnTargetScore'] == 5)
                                        & (poolPUS7_break_pair_df['TreeEditDistance'] <= pool_break_pair_ted_threshold) 
                                        & (poolPUS7_break_pair_df['OffTargetScore'] <= pool_break_pair_off_threshold)
                                        ]
pool_good_make_pair_df = poolPUS7_make_pair_df.loc[(poolPUS7_make_pair_df['OnTargetScore'] == 5)
                                        &  (poolPUS7_make_pair_df['TreeEditDistance'] <= pool_make_pair_ted_threshold) 
                                        & (poolPUS7_make_pair_df['OffTargetScore'] <= pool_make_pair_off_threshold)
                                        ]

print(f"Number of good break pair: {len(pool_good_break_pair_df)}")
print(f"Number of good make pair: {len(pool_good_make_pair_df)}")

pool_good_break_pair_df.to_csv("pool_good_breakpair_5mer.csv", index=False)
pool_good_make_pair_df.to_csv("pool_good_makepair_5mer.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 628 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 495 rows.
Creating DataFrame for logic type: 'make_pair' with 133 rows.
Number of break pair: 247
Number of make pair: 50
Number of good break pair: 28
Number of good make pair: 7

Analysis complete. Results saved.


## Unpaired2 Accessibility

In [11]:
unmod_access_psi="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/unpaired2_pairing/pairing_status_flip_analysis.csv"

unmod_access_psi_df = pd.read_csv(unmod_access_psi)

column_mapping = {
    'mutant_ids': unmod_access_psi_df['name'],
    'parent_seqs': unmod_access_psi_df['original_seq'],
    'mutant_seqs': unmod_access_psi_df['mutated_seq'],
    'parent_dbs': unmod_access_psi_df['original_dot_bracket'],
    'mutant_dbs': unmod_access_psi_df['dot_bracket_mut'],
    'goal_types': unmod_access_psi_df['strategy'],
    'target_regions_1': unmod_access_psi_df['target_pos_range']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

break_pair_df = dfs_by_logic.get('break_pair')
make_pair_df = dfs_by_logic.get('make_pair')

print(f"Number of break pair: {len(break_pair_df)}")
print(f"Number of make pair: {len(make_pair_df)}")

break_pair_ted_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_ted_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

break_pair_off_threshold = break_pair_df['TreeEditDistance'].quantile(0.03)
make_pair_off_threshold = make_pair_df['TreeEditDistance'].quantile(0.1)

good_break_pair_df = break_pair_df.loc[(break_pair_df['OnTargetScore'] == 3)
                                       & (break_pair_df['TreeEditDistance'] <= break_pair_ted_threshold) 
                                       & (break_pair_df['OffTargetScore'] <= break_pair_off_threshold)
                                       ]
good_make_pair_df = make_pair_df.loc[(make_pair_df['OnTargetScore'] == 3)
                                     & (make_pair_df['TreeEditDistance'] <= make_pair_ted_threshold) 
                                     & (make_pair_df['OffTargetScore'] <= make_pair_off_threshold)
                                     ]

print(f"Number of good break pair: {len(good_break_pair_df)}")

good_break_pair_df.to_csv("unmod_good_breakpair_unpaired2.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 545 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 417 rows.
Creating DataFrame for logic type: 'make_pair' with 128 rows.
Number of break pair: 417
Number of make pair: 128
Number of good break pair: 12

Analysis complete. Results saved.


In [12]:
pool_access="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/unpaired2_pairing/pairing_status_flip_analysis_pool1.csv"

pool_access_df = pd.read_csv(pool_access)

column_mapping = {
    'mutant_ids': pool_access_df['name'],
    'parent_seqs': pool_access_df['original_seq'],
    'mutant_seqs': pool_access_df['mutated_seq'],
    'parent_dbs': pool_access_df['original_dot_bracket'],
    'mutant_dbs': pool_access_df['dot_bracket_mut'],
    'goal_types': pool_access_df['strategy'],
    'target_regions_1': pool_access_df['target_pos_range']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('Logic_Type_Applied'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

pool_break_pair_df = dfs_by_logic.get('break_pair')
pool_make_pair_df = dfs_by_logic.get('make_pair')

# The concise, single-line solution
poolPUS7_make_pair_df = pool_make_pair_df[pool_make_pair_df['mutant_id'].isin(PUS7_union_df['chr'])]
poolPUS7_break_pair_df = pool_break_pair_df[pool_break_pair_df['mutant_id'].isin(unuar_df['chr'])]

print(f"Number of break pair: {len(poolPUS7_break_pair_df)}")
print(f"Number of make pair: {len(poolPUS7_make_pair_df)}")

pool_break_pair_ted_threshold = poolPUS7_break_pair_df['TreeEditDistance'].quantile(0.5)
pool_make_pair_ted_threshold = poolPUS7_make_pair_df['TreeEditDistance'].quantile(0.5)

pool_break_pair_off_threshold = poolPUS7_break_pair_df['OffTargetScore'].quantile(0.5)
pool_make_pair_off_threshold = poolPUS7_make_pair_df['OffTargetScore'].quantile(0.5)

pool_good_break_pair_df = poolPUS7_break_pair_df.loc[(poolPUS7_break_pair_df['OnTargetScore'] == 3)
                                        & (poolPUS7_break_pair_df['TreeEditDistance'] <= pool_break_pair_ted_threshold) 
                                        & (poolPUS7_break_pair_df['OffTargetScore'] <= pool_break_pair_off_threshold)
                                        ]
pool_good_make_pair_df = poolPUS7_make_pair_df.loc[(poolPUS7_make_pair_df['OnTargetScore'] == 3)
                                        &  (poolPUS7_make_pair_df['TreeEditDistance'] <= pool_make_pair_ted_threshold) 
                                        & (poolPUS7_make_pair_df['OffTargetScore'] <= pool_make_pair_off_threshold)
                                        ]

print(f"Number of good break pair: {len(pool_good_break_pair_df)}")
print(f"Number of good make pair: {len(pool_good_make_pair_df)}")

pool_good_break_pair_df.to_csv("pool_good_breakpair_unpaired2.csv", index=False)
pool_good_make_pair_df.to_csv("pool_good_makepair_unpaired2.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


DataFrame loaded successfully.
Starting analysis of 604 entries...
Analysis function finished.
Creating DataFrame for logic type: 'break_pair' with 419 rows.
Creating DataFrame for logic type: 'make_pair' with 185 rows.
Number of break pair: 196
Number of make pair: 84
Number of good break pair: 33
Number of good make pair: 17

Analysis complete. Results saved.


## Structure Mutation

In [13]:
pool_structure="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/structure_mutation/all_mutagenesis_results.csv"

pool_structure_df = pd.read_csv(pool_structure)

#display(pool_structure_df.head())

# filter for PUS7 Union
pool_structure_df['name'] = pool_structure_df['original_filename'].str.removesuffix('.fold')
poolPUS7_structure_df = pool_structure_df[pool_structure_df['name'].isin(PUS7_union_df['chr'])]

print(f"Number of mutations in PUS7 union: {len(poolPUS7_structure_df)}")

# Remove duplicated total seqeunces that match an up or downstream mutant
priority_order = [
    'upstream_disrution',
    'downstream_disruption',
    'total_disruption',
    'upstream_compensation',
    'downstream_compensation',
    'total_compensation'
]

poolPUS7_structure_df['goal_type_cat'] = pd.Categorical(
    poolPUS7_structure_df['goal_type'],
    categories=priority_order,
    ordered=True
)

filtered_poolPUS7_structure_df = poolPUS7_structure_df.sort_values('goal_type_cat').drop_duplicates(
    subset='mutant_seq', 
    keep='first'
)

print(f"Number of mutations in filtered PUS7 union: {len(filtered_poolPUS7_structure_df)}")


column_mapping = {
    'mutant_ids': filtered_poolPUS7_structure_df['mutant_id'],
    'parent_seqs': filtered_poolPUS7_structure_df['parent_seq'],
    'mutant_seqs': filtered_poolPUS7_structure_df['mutant_seq'],
    'parent_dbs': filtered_poolPUS7_structure_df['parent_db'],
    'mutant_dbs': filtered_poolPUS7_structure_df['mutant_db'],
    'goal_types': filtered_poolPUS7_structure_df['goal_type'],
    'target_regions_1': filtered_poolPUS7_structure_df['target_mut_region']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('goal_type'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

down_disr_df = dfs_by_logic.get('downstream_disruption')
down_comp_df = dfs_by_logic.get('downstream_compensation')
up_disr_df = dfs_by_logic.get('upstream_disruption')
up_comp_df = dfs_by_logic.get('upstream_compensation')
total_disr_df = dfs_by_logic.get('total_disruption')
total_comp_df = dfs_by_logic.get('total_compensation')

down_disrupt_on_threshold = down_disr_df['OnTargetScore'].quantile(0.5)
down_disrupt_ted_threshold = down_disr_df['TreeEditDistance'].quantile(0.5)
down_disrupt_off_threshold = down_disr_df['OffTargetScore'].quantile(0.3)

up_disrupt_on_threshold = up_disr_df['OnTargetScore'].quantile(0.5)
up_disrupt_ted_threshold = up_disr_df['TreeEditDistance'].quantile(0.5)
up_disrupt_off_threshold = up_disr_df['OffTargetScore'].quantile(0.3)

tot_disrupt_on_threshold = total_disr_df['OnTargetScore'].quantile(0.5)
tot_disrupt_ted_threshold = total_disr_df['TreeEditDistance'].quantile(0.5)
tot_disrupt_off_threshold = total_disr_df['OffTargetScore'].quantile(0.3)


good_down_disr_df = down_disr_df.loc[(down_disr_df['OnTargetScore'] <= down_disrupt_on_threshold)
                                        & (down_disr_df['TreeEditDistance'] <= down_disrupt_ted_threshold) 
                                        & (down_disr_df['OffTargetScore'] <= down_disrupt_off_threshold)
                                        ]
good_up_disr_df = up_disr_df.loc[(up_disr_df['OnTargetScore'] <= up_disrupt_on_threshold)
                                        & (up_disr_df['TreeEditDistance'] <= up_disrupt_ted_threshold) 
                                        & (up_disr_df['OffTargetScore'] <= up_disrupt_off_threshold)
                                        ]
good_tot_disr_df = total_disr_df.loc[(total_disr_df['OnTargetScore'] <= tot_disrupt_on_threshold)
                                        & (total_disr_df['TreeEditDistance'] <= tot_disrupt_ted_threshold) 
                                        & (total_disr_df['OffTargetScore'] <= tot_disrupt_off_threshold)
                                        ]
print()
print(f"Number of good downstream disruptions: {len(good_down_disr_df)}")
print(f"Number of good upstream disruptions: {len(good_up_disr_df)}")
print(f"Number of good total disruptions: {len(good_tot_disr_df)}")

successful_down_disr = set(
    good_down_disr_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True)
)
final_down_comp_df = down_comp_df[
    down_comp_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True).isin(successful_down_disr)
].copy()


successful_up_disr = set(
    good_up_disr_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True)
)
final_up_comp_df = up_comp_df[
    up_comp_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True).isin(successful_up_disr)
].copy()


successful_tot_disr = set(
    good_tot_disr_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True)
)
final_tot_comp_df = total_comp_df[
    total_comp_df['mutant_id'].str.replace(r'_disruption$|_compensation$', '', regex=True).isin(successful_tot_disr)
].copy()

print()
print(f"Number of downstream compensations: {len(final_down_comp_df)}")
print(f"Number of upstream compensations: {len(final_up_comp_df)}")
print(f"Number of total compensations: {len(final_tot_comp_df)}")

down_comp_on_threshold = final_down_comp_df['OnTargetScore'].quantile(0.5)
down_comp_ted_threshold = final_down_comp_df['TreeEditDistance'].quantile(0.5)
down_comp_off_threshold = final_down_comp_df['OffTargetScore'].quantile(0.5)

up_comp_on_threshold = final_up_comp_df['OnTargetScore'].quantile(0.5)
up_comp_ted_threshold = final_up_comp_df['TreeEditDistance'].quantile(0.5)
up_comp_off_threshold = final_up_comp_df['OffTargetScore'].quantile(0.5)

tot_comp_on_threshold = final_tot_comp_df['OnTargetScore'].quantile(0.5)
tot_comp_ted_threshold = final_tot_comp_df['TreeEditDistance'].quantile(0.5)
tot_comp_off_threshold = final_tot_comp_df['OffTargetScore'].quantile(0.5)


good_down_comp_df = final_down_comp_df.loc[(final_down_comp_df['OnTargetScore'] <= down_comp_on_threshold)
                                        & (final_down_comp_df['TreeEditDistance'] <= down_comp_ted_threshold) 
                                        & (final_down_comp_df['OffTargetScore'] <= down_comp_off_threshold)
                                        ]
good_up_comp_df = final_up_comp_df.loc[(final_up_comp_df['OnTargetScore'] <= up_comp_on_threshold)
                                        & (final_up_comp_df['TreeEditDistance'] <= up_comp_ted_threshold) 
                                        & (final_up_comp_df['OffTargetScore'] <= up_comp_off_threshold)
                                        ]
good_tot_comp_df = final_tot_comp_df.loc[(final_tot_comp_df['OnTargetScore'] <= tot_comp_on_threshold)
                                        & (final_tot_comp_df['TreeEditDistance'] <= tot_comp_ted_threshold) 
                                        & (final_tot_comp_df['OffTargetScore'] <= tot_comp_off_threshold)
                                        ]
print()
print(f"Number of good downstream compensations: {len(good_down_comp_df)}")
print(f"Number of good upstream compensations: {len(good_up_comp_df)}")
print(f"Number of good total compensations: {len(good_tot_comp_df)}")

# Specific sites to include
sites = ["RHBDD2", "IGF2BP1", "FIS1", "HDAC6", "EIF5"]
specsite_structure_df = results_df[results_df['mutant_id'].str.contains('|'.join(sites), case=False, na=False)]
#specsite_structure_df = specsite_structure_df.query("goal_type != 'total_compensation'")

specsite_structure_df.to_csv("pool_specific_struct_mut.csv", index=False)
good_down_disr_df.to_csv("pool_good_down_disruption.csv", index=False)
good_up_disr_df.to_csv("pool_good_up_disruption.csv", index=False)
good_tot_disr_df.to_csv("pool_good_tot_disruption.csv", index=False)
good_down_comp_df.to_csv("pool_good_down_compensation.csv", index=False)
good_up_comp_df.to_csv("pool_good_up_compensation.csv", index=False)
good_tot_comp_df.to_csv("pool_good_tot_compensation.csv", index=False)
print(f"\nAnalysis complete. Results saved.")


Number of mutations in PUS7 union: 1516
Number of mutations in filtered PUS7 union: 1343
DataFrame loaded successfully.
Starting analysis of 1343 entries...


C:\Users\rmrex\AppData\Local\Temp\ipykernel_20396\1465844968.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poolPUS7_structure_df['goal_type_cat'] = pd.Categorical(


Analysis function finished.
Creating DataFrame for logic type: 'downstream_compensation' with 239 rows.
Creating DataFrame for logic type: 'downstream_disruption' with 260 rows.
Creating DataFrame for logic type: 'total_compensation' with 184 rows.
Creating DataFrame for logic type: 'total_disruption' with 225 rows.
Creating DataFrame for logic type: 'upstream_compensation' with 222 rows.
Creating DataFrame for logic type: 'upstream_disruption' with 213 rows.

Number of good downstream disruptions: 47
Number of good upstream disruptions: 29
Number of good total disruptions: 34

Number of downstream compensations: 42
Number of upstream compensations: 29
Number of total compensations: 26

Number of good downstream compensations: 11
Number of good upstream compensations: 12
Number of good total compensations: 9

Analysis complete. Results saved.


## Stability

In [14]:
pool_stability="C:/Users/rmrex/OneDrive - Stanford/Martinez Lab/R/Pool2_Mutagenesis/Abridged/MFE_alterations/significant_mfe_mutants_pool1.csv"

pool_stability_df = pd.read_csv(pool_stability)

# filter for PUS7 Union
pool_stability_df['name'] = pool_stability_df['original_filename'].str.removesuffix('.fold')
poolPUS7_stability_df = pool_stability_df[pool_stability_df['name'].isin(PUS7_union_df['chr'])]


# Add in target ranges
# upstream = 56-63
# downstram = 69-81
# total = 56-63, 69-81
conditions = [
    poolPUS7_stability_df['type'].str.contains('total', na=False),
    poolPUS7_stability_df['type'].str.contains('downstream', na=False),
    poolPUS7_stability_df['type'].str.contains('upstream', na=False)
]
# must match order of conditions list
choices = [
    '56-63, 69-81',
    '69-81',
    '56-63'
]
poolPUS7_stability_df['target_region'] = np.select(conditions, choices, default=None)

print(f"Number of mutations in PUS7 union: {len(poolPUS7_stability_df)}")

# Remove duplicated total seqeunces that match an up or downstream mutant
# redundant_sequences = set(
#     poolPUS7_stability_df.loc[
#         poolPUS7_stability_df['type'].str.contains('up|down', na=False), 'mutant_seq'
#     ]
# )
# mask_to_keep = ~(
#     poolPUS7_stability_df['type'].str.contains('total', na=False) &
#     poolPUS7_stability_df['mutant_seq'].isin(redundant_sequences)
# )

# filtered_poolPUS7_stability_df = poolPUS7_stability_df[mask_to_keep].copy()

priority_order = [
    'upstream_stabilizing',
    'downstream_stabilizing',
    'total_stabilizing',
    'upstream_destabilizing',
    'downstream_destabilizing',
    'total_destabilizing'
]

poolPUS7_stability_df['goal_type_cat'] = pd.Categorical(
    poolPUS7_stability_df['type'],
    categories=priority_order,
    ordered=True
)

filtered_poolPUS7_stability_df = poolPUS7_stability_df.sort_values('goal_type_cat').drop_duplicates(
    subset='mutant_seq', 
    keep='first'
)

print(f"Number of mutations in filtered PUS7 union: {len(filtered_poolPUS7_stability_df)}")
#display(poolPUS7_stability_df.head())


column_mapping = {
    'mutant_ids': filtered_poolPUS7_stability_df['name'],
    'parent_seqs': filtered_poolPUS7_stability_df['sequence'],
    'mutant_seqs': filtered_poolPUS7_stability_df['mutant_seq'],
    'parent_dbs': filtered_poolPUS7_stability_df['original_db'],
    'mutant_dbs': filtered_poolPUS7_stability_df['mutant_db'],
    'goal_types': filtered_poolPUS7_stability_df['type'],
    'target_regions_1': filtered_poolPUS7_stability_df['target_region']
}

print("DataFrame loaded successfully.")

results_df = run_analysis_engine(**column_mapping)

dfs_by_logic = {}

for name, group in results_df.groupby('goal_type'):
    print(f"Creating DataFrame for logic type: '{name}' with {len(group)} rows.")
    dfs_by_logic[name] = group

down_stab_df = dfs_by_logic.get('downstream_stabilizing')
down_destab_df = dfs_by_logic.get('downstream_destabilizing')
up_stab_df = dfs_by_logic.get('upstream_stabilizing')
up_destab_df = dfs_by_logic.get('upstream_destabilizing')
total_stab_df = dfs_by_logic.get('total_stabilizing')
total_destab_df = dfs_by_logic.get('total_destabilizing')

dataframes_to_check = {
    "down_stab_df": down_stab_df,
    "down_destab_df": down_destab_df,
    "up_stab_df": up_stab_df,
    "up_destab_df": up_destab_df,
    "total_stab_df": total_stab_df,
    "total_destab_df": total_destab_df
}

print("--- DataFrame Lengths ---")
for name, df in dataframes_to_check.items():
    if df is not None:
        #print()
        print(f"{name}: {len(df)} rows")

        ted_threshold = df['TreeEditDistance'].quantile(0.25)
        off_threshold = df['OffTargetScore'].quantile(0.25)

        good_df = df.loc[(df['TreeEditDistance'] <= ted_threshold) 
                                & (df['OffTargetScore'] <= off_threshold)
                                ]
        print(f"Number of good stability mutations in {name}: {len(good_df)}")

        output_filename = f"pool_good_{name}.csv"
        good_df.to_csv(output_filename, index=False)

    else:
        print(f"{name}: Does not exist (None)")

print(f"\nAnalysis complete. Results saved.")

Number of mutations in PUS7 union: 1290
Number of mutations in filtered PUS7 union: 1061
DataFrame loaded successfully.
Starting analysis of 1061 entries...
Analysis function finished.
Creating DataFrame for logic type: 'downstream_destabilizing' with 237 rows.
Creating DataFrame for logic type: 'downstream_stabilizing' with 172 rows.
Creating DataFrame for logic type: 'total_destabilizing' with 162 rows.
Creating DataFrame for logic type: 'total_stabilizing' with 97 rows.
Creating DataFrame for logic type: 'upstream_destabilizing' with 214 rows.
Creating DataFrame for logic type: 'upstream_stabilizing' with 179 rows.
--- DataFrame Lengths ---
down_stab_df: 172 rows
Number of good stability mutations in down_stab_df: 31
down_destab_df: 237 rows
Number of good stability mutations in down_destab_df: 40
up_stab_df: 179 rows
Number of good stability mutations in up_stab_df: 34
up_destab_df: 214 rows
Number of good stability mutations in up_destab_df: 34
total_stab_df: 97 rows
Number of goo

C:\Users\rmrex\AppData\Local\Temp\ipykernel_20396\2600160809.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poolPUS7_stability_df['target_region'] = np.select(conditions, choices, default=None)
C:\Users\rmrex\AppData\Local\Temp\ipykernel_20396\2600160809.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poolPUS7_stability_df['goal_type_cat'] = pd.Categorical(
